In [1]:
# TEST SET (reloading independent code fragment for now sampled excel)

# Import CNN Zero-Shot & other necessary stuff
from transformers import pipeline  # It takes  time here
# For data input and data cleaning
import pandas as pd
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from numpy import NaN
import time

# Available directories (input yours for personal use)
cep_data_dir = "/home/aan0709@tmme/pcu-research-mapping/data/"

working_dir = "TEST/"    # CHANGE WORKING DIRECTORY INSIDE IEEE XPLORE FOLDER

# For data importing
def openExcel(excel_name, sheet_name, directory):
    df = pd.read_excel(directory + excel_name + ".xlsx", sheet_name=sheet_name)
    return df

def cleanIndexes(df):
    temp_ls = df.columns
    for i in df.columns:
        if i == "abstract": break
        df.drop(columns=[i], inplace=True)
    return df


# Select INPUT EXCEL
file_name = "TESTSET_DC-DC_answered"    # -> I have edited the Affiliations and the Keywords part

df = openExcel(file_name, "Sheet1", cep_data_dir + working_dir)

df = cleanIndexes(df)
df.reset_index(drop=True, inplace=True)

In [2]:
start = time.time()
classifierGPU0 = pipeline("zero-shot-classification", model='facebook/bart-large-mnli', device=0)
end = time.time()
print("seconds elapsed: ", end-start)

seconds elapsed:  10.904866695404053


In [3]:
Author_Keywords = "index_terms.author_terms.terms"
IEEE_Keywords = "index_terms.ieee_terms.terms"

# Keep only those w/ IEEE Key
ieee_key = "DC-DC power converters"
df_new = pd.DataFrame()
for i in range(len(df["Manual Judgement"])):
    if ieee_key in df.loc[i, IEEE_Keywords]:
        df_new = df_new.append(df.loc[i])

keeping = len(df_new)
df_new.reset_index(drop=True, inplace=True)
print(keeping)

# Those that will be appended in the end will have been predefined as wrong
for  i in range(len(df)):
    flag = True
    for j in range(len(df_new)):
        if df_new.loc[j, "abstract"] == df.loc[i, "abstract"]:
            flag = False
            break
    if flag:
        df_new = df_new.append(df.loc[i])
        df_new.reset_index(drop=True, inplace=True)

df_new["ZeroShot Judgement"] = NaN
for i in range(len(df_new)-1, keeping-1, -1):
    df_new.loc[i, "ZeroShot Judgement"] = 0

df = df_new

15


In [3]:
Author_Keywords = "index_terms.author_terms.terms"
IEEE_Keywords = "index_terms.ieee_terms.terms"

# count 0s and 1s:
count1 = 0
count_key = 0
ieee_key = "DC-DC power converters"

for i in range(len(df["Manual Judgement"])):
    if str(int(df.loc[i, "Manual Judgement"])) == '1':
        count1 += 1
        if ieee_key in df.loc[i, IEEE_Keywords]:
            count_key += 1

print("DC-DC power converters: %d/50, w/ key %d/%d" % (count1, count_key, count1))

DC-DC power converters: 19/50, w/ key 13/19


In [14]:
# TEST 0

positive_outcome = "dc-dc converter dc/dc converter"
# negative_outcome = "not traction inverter"

candidate_labels = [positive_outcome] # , negative_outcome]
hypothesis_template = "The research is about {}?"    # Categorization question

In [23]:
# TEST 1

positive_outcome = "dc-dc converter dc/dc converter LLC resonant dc-dc converter double active bridge DAB dc-dc converter"
negative_outcome = "pulse width modulation PWM converter AC-DC AC/DC converter DC-AC DC/AC converter AC-AC AC/AC converter wireless converter"

candidate_labels = ["dc-dc converter dc/dc converter", "LLC resonant dc-dc converter", "double active bridge DAB dc-dc converter",
                    "pulse width modulation PWM converter", "AC-DC AC/DC converter", "DC-AC DC/AC converter", "AC-AC AC/AC converter", "wireless converter"]

hypothesis_template = "The research is about {}?"    # Categorization question

In [4]:
# Method 2 - 

positive_outcome = "dc-dc dc/dc converter LLC resonant converter double active bridge DAB converter"
negative_outcome = "pulse width modulation PWM converter AC-DC AC/DC converter DC-AC DC/AC converter AC-AC AC/AC converter wireless converter"

candidate_labels = ["dc-dc dc/dc converter", "LLC resonant converter", "double active bridge DAB converter",
                    "pulse width modulation PWM", "AC-DC AC/DC converter", "DC-AC DC/AC converter", "AC-AC AC/AC converter", "wireless"]

hypothesis_template = "The research is about {}?"    # Categorization question

In [7]:
from stop_words import get_stop_words
from nltk.corpus import stopwords

percentages = []
for perc_loop in range(35, 100, 5):

    en_stopwords = []
    en_stopwords = list(get_stop_words('en'))         #About 900 stopwords
    nltk_words = list(stopwords.words('english'))     #About 150 stopwords
    en_stopwords.extend(nltk_words)

    lemma = WordNetLemmatizer()

    Author_Keywords = "index_terms.author_terms.terms"
    IEEE_Keywords = "index_terms.ieee_terms.terms"

    zero_shot_judgement = []
    # df["ZeroShot Judgement"] = NaN

    count_all = len(df)
    count_diff = 0

    start = time.time()

    final_judgement_dict = {}

    # for i in range(len(df)):
    for i in range(len(df)):
        data = ""

        # Where to search
        if isinstance(df.loc[i, "abstract"], str):
            data += df.loc[i, "abstract"]
        if isinstance(df.loc[i, "title"], str):
            data += " " + df.loc[i, "title"]
        exception_to_str = df.loc[i, IEEE_Keywords]
        if isinstance(exception_to_str, str):
            exception_to_str = " ".join(df.loc[i, IEEE_Keywords].split(";"))
            data += " " + exception_to_str
        exception_to_str = df.loc[i, Author_Keywords]
        if isinstance(exception_to_str, str):
            exception_to_str = " ".join(df.loc[i, Author_Keywords].split(";"))
            data += " " + exception_to_str
            printing_data = data

        # Data Cleaning with NLTK, NumPy
        tokens = regexp_tokenize(data, pattern=r"\s|[\.,;'()]", gaps=True)
        words = []
        for k in tokens:
            if k not in en_stopwords:
                k = lemma.lemmatize(k)
            words.append(k)
        data = " ".join(words)

        # print(data)

        # Here the data has been cleaned
        input_text = data

        # Rienforcing the data with keyword
        tokens = regexp_tokenize(data, pattern=r"\s|[\.,;'()]", gaps=True)
        for j in tokens:
            if j in candidate_labels[0]:
                data += j

        start_judge = time.time()
        # Extract the probabilities from the CNN
        final_judgement_dict = classifierGPU0(input_text, candidate_labels, hypothesis_template=hypothesis_template, multi_label=True)

        end_judge = time.time()

        if final_judgement_dict["labels"][0] in positive_outcome and final_judgement_dict["scores"][0] > perc_loop/100.0:
            zero_shot_judge = 1
        else:
            zero_shot_judge = 0
        # print(zero_shot_judge)
        zero_shot_judgement.append(zero_shot_judge)
        df.loc[i, "ZeroShot Judgement"] = float(zero_shot_judge)

        # print(data)


    while len(zero_shot_judgement) < len(df):
        zero_shot_judgement.append(0)

    end = time.time()

    # Print differences
    for i in range(len(df)):
        if df.loc[i, "ZeroShot Judgement"] != df.loc[i, "Manual Judgement"]:
            count_diff += 1

            """print(df.loc[i, "Manual Judgement"])
            print("----- DECISION ----- :")
            print(i)
            if str(zero_shot_judgement[i]) == "1":
                print("-- It is included --")
            else:
                print("-- It is NNNOOOTTT included --")    # Visual exageration to easily spot answer
            print()
            if isinstance(df.loc[i, Author_Keywords], str): auth_keys = df.loc[i, Author_Keywords].split(";")
            else: auth_keys = ""
            if isinstance(df.loc[i, IEEE_Keywords], str): ieee_keys = df.loc[i, IEEE_Keywords].split(";")
            else: ieee_keys = ""
            print("Abstract :")
            print(df.loc[i, "abstract"], "\n")
            print("Document Title :")
            print(df.loc[i, "title"], "\n")
            print("Author Keywords :")
            [print("-> ", x) for x in auth_keys]
            print()
            print("IEEE Keywords :")
            [print("-> ", x) for x in ieee_keys]"""


    count = 0
    for i in range(len(df)):
        if df.loc[i, "ZeroShot Judgement"] == "1":
            count += 1

    percentages.append((count_all-count_diff)/count_all)
    print(perc_loop/100)
    m, s = divmod(end - start, 60)
    print("Time: %d min %0.3f sec" % (m,s))
    print("From %d papers, the %d are CORRECT and the %d are WRONG ( %.2f percent correct )\n" % (count_all, count_all-count_diff, count_diff, (100*(count_all-count_diff)/count_all)))

0.35
Time: 0 min 13.755 sec
From 50 papers, the 40 are CORRECT and the 10 are WRONG ( 80.00 percent correct )

0.4
Time: 0 min 13.973 sec
From 50 papers, the 40 are CORRECT and the 10 are WRONG ( 80.00 percent correct )

0.45
Time: 0 min 13.600 sec
From 50 papers, the 41 are CORRECT and the 9 are WRONG ( 82.00 percent correct )

0.5
Time: 0 min 13.423 sec
From 50 papers, the 43 are CORRECT and the 7 are WRONG ( 86.00 percent correct )

0.55
Time: 0 min 12.598 sec
From 50 papers, the 42 are CORRECT and the 8 are WRONG ( 84.00 percent correct )

0.6
Time: 0 min 13.377 sec
From 50 papers, the 42 are CORRECT and the 8 are WRONG ( 84.00 percent correct )

0.65
Time: 0 min 13.335 sec
From 50 papers, the 42 are CORRECT and the 8 are WRONG ( 84.00 percent correct )

0.7
Time: 0 min 14.153 sec
From 50 papers, the 42 are CORRECT and the 8 are WRONG ( 84.00 percent correct )

0.75
Time: 0 min 13.591 sec
From 50 papers, the 42 are CORRECT and the 8 are WRONG ( 84.00 percent correct )

0.8
Time: 0 